In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git`
!pip install -q pillow

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import ViltProcessor, ViltForImageAndTextRetrieval
import torch
from PIL import Image
import pandas as pd
from tqdm.notebook import tqdm
import os

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-coco")
model = ViltForImageAndTextRetrieval.from_pretrained("dandelin/vilt-b32-finetuned-coco")
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/446M [00:00<?, ?B/s]

ViltForImageAndTextRetrieval(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_feature

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/Project2_4/dataset.csv')

In [ ]:
def get_label_scores(image_path, labels):
  image = Image.open(image_path).convert('RGB')
  scores = {}
  for label in labels:
    encoding = processor(image, label, return_tensors="pt").to(device)
    outputs = model(**encoding)
    score = outputs.logits[:, 0].item()
    scores[label] = score
  return scores

results = []

for index, row in tqdm(dataset.iterrows(), total=dataset.shape[0]):
  image_path = row['image']
  labels = [row['label1'], row['label2'], row['label3']]
  scores = get_label_scores(image_path, labels)
  ranked_labels = sorted(scores.items(), key=lambda item: item[1], reverse=True)
  ranked_dict = {ranked_labels[i][0]: i+1 for i in range(len(ranked_labels))}

  results.append({
      'image': image_path,
      'label1': labels[0],
      'label2': labels[1],
      'label3': labels[2],
      'label1_rank': ranked_dict[labels[0]],
      'label2_rank': ranked_dict[labels[1]],
      'label3_rank': ranked_dict[labels[2]]
  })

results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/Project2_4/VILT_lables.csv', index=False)
results_df.head()

  0%|          | 0/36 [00:00<?, ?it/s]

,image,label1,label2,label3,label1_rank,label2_rank,label3_rank
0,/content/drive/MyDrive/Project2_4/images/new1.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,1,3,2
1,/content/drive/MyDrive/Project2_4/images/new2.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,1,3,2
2,/content/drive/MyDrive/Project2_4/images/new3.png,The phone on the left is new,The phone on the left is newer,The phone on the left is the newest,1,3,2
3,/content/drive/MyDrive/Project2_4/images/short...,The woman's hair is short,The woman's hair is shorter,The woman's hair is the shortest,3,2,1
4,/content/drive/MyDrive/Project2_4/images/short...,The woman's hair is short,The woman's hair is shorter,The woman's hair is the shortest,3,2,1


In [ ]:
# @title label1_rank

from matplotlib import pyplot as plt
results_df['label1_rank'].plot(kind='hist', bins=20, title='label1_rank')
plt.gca().spines[['top', 'right',]].set_visible(False)